In [1]:
import pandas as pd

In [8]:
data=pd.read_feather('/Users/mstudio/repository/research/program-l/data/feather/thread.feather')

In [31]:
data.sample(100)[['thread_name', 'account', 'date', 'body']].to_csv('sample.csv', index=False, header=False, sep='\t')

In [35]:
data

,thread_id,thread_name,body,account,url,date,longevity
0,0,virtual-machines,"[Hi Guys, Is Virtual accessible? What vm softw...","[blindwiz, soronel.haetir, len, sonfire11]",[https://www.freelists.org/post/program-l/virt...,"[2014-10-03T22:40:50.000000000, 2014-10-03T23:...",0 days 00:58:47
1,1,Tools-Must-Have,"[A pleasant day to everyone, Can someone pleas...","[dayugdog, soronel.haetir, ofbgmail, jdashiel,...",[https://www.freelists.org/post/program-l/Tool...,"[2014-10-21T11:44:48.000000000, 2014-10-21T14:...",2 days 14:58:37
2,2,Working-With-Diagrams,"[Hello everyone, Are there most effectiv ways ...","[dayugdog, nomad42]",[https://www.freelists.org/post/program-l/Work...,"[2015-01-18T07:56:29.000000000, 2015-01-18T16:...",0 days 08:40:44
3,3,SSIS-SSRS-Designers-with-Jaws-15,[I ’ m teaching a JAWS user SSIS ( SQL Server ...,"[billiam904, ofbgmail]",[https://www.freelists.org/post/program-l/SSIS...,"[2015-03-16T12:37:15.000000000, 2015-03-16T15:...",0 days 02:31:54
4,4,Side-by-side-DISfiguration-LOL,"[Hi guys, I've got a question about a new prob...","[mfox32322, soronel.haetir, mfox32322, ofbgmai...",[https://www.freelists.org/post/program-l/Side...,"[2015-03-17T21:12:40.000000000, 2015-03-17T23:...",1 days 09:11:32
...,...,...,...,...,...,...,...
8339,8339,Python-XML-accessing-data-in-sub-elements,"[Hi, Here is my code so far: import xml. etree...","[reynoldsdavid46, travis, kperry, kperry, teac...",[https://www.freelists.org/post/program-l/Pyth...,"[2022-11-07T16:28:49.000000000, 2022-11-07T17:...",0 days 05:08:24
8340,8340,Git-Article-Outlining-Typical-Git-Workflow,"[Hi, This is a nice article that illustrates t...","[jhomme, lewislwood]",[https://www.freelists.org/post/program-l/Git-...,"[2022-11-07T13:56:16.000000000, 2022-11-07T18:...",0 days 04:21:50
8341,8341,A-Button-That-Copies-Code-To-The-Clipboard,"[Hi, Does anyone know where to find instructio...","[jhomme, lewislwood]",[https://www.freelists.org/post/program-l/A-Bu...,"[2022-11-08T14:50:11.000000000, 2022-11-08T15:...",0 days 00:19:44
8342,8342,TeX-Hour-Thu-10-Nov-arXiv-access-and-the-TeX-M...,"[Hi Tomorrow's TeX Hour is "" arXiv access the ...","[jfine2358, jfine2358]",[https://www.freelists.org/post/program-l/TeX-...,"[2022-11-09T19:53:18.000000000, 2022-11-09T19:...",0 days 00:00:37


In [20]:
data.dtypes

thread_id                int64
thread_name             object
body                    object
account                 object
url                     object
date                    object
longevity      timedelta64[ns]
dtype: object

In [14]:
data[['account', 'date']].loc[0, 'date']

array(['2014-10-03T22:40:50.000000000', '2014-10-03T23:05:36.000000000',
       '2014-10-03T23:11:36.000000000', '2014-10-03T23:39:37.000000000'],
      dtype='datetime64[ns]')

In [15]:
data[['account', 'date']].loc[0, 'date'].min()

numpy.datetime64('2014-10-03T22:40:50.000000000')

In [16]:
data[['account', 'date']].loc[0, 'date'].max()

numpy.datetime64('2014-10-03T23:39:37.000000000')

In [7]:
data.loc[0, 'related_posts']

'https://www.freelists.org/post/program-l/otiis-being-weird; https://www.freelists.org/post/program-l/otiis-being-weird,1; https://www.freelists.org/post/program-l/otiis-being-weird,2; https://www.freelists.org/post/program-l/otiis-being-weird,3; https://www.freelists.org/post/program-l/otiis-being-weird,4; https://www.freelists.org/post/program-l/otiis-being-weird,5; https://www.freelists.org/post/program-l/otiis-being-weird,6; https://www.freelists.org/post/program-l/otiis-being-weird,7; https://www.freelists.org/post/program-l/otiis-being-weird,8; https://www.freelists.org/post/program-l/otiis-being-weird,9; https://www.freelists.org/post/program-l/otiis-being-weird,10; https://www.freelists.org/post/program-l/otiis-being-weird,11; https://www.freelists.org/post/program-l/otiis-being-weird,12; https://www.freelists.org/post/program-l/otiis-being-weird,13; https://www.freelists.org/post/program-l/otiis-being-weird,15; https://www.freelists.org/post/program-l/otiis-being-weird,16; htt

In [3]:
data

,subject,from,to,date,gmt_offset,date_utc,post_url,followup_url,references_url,related_posts,references_count,followup_count,related_posts_count,new_utc,body
0,Re: ot--iis being weird. - program-l - FreeLists,"""Tyler Littlefield"" <compgeek13@xxxxxxxxx>",<program-l@xxxxxxxxxxxxx>,"Sun, 31 Oct 2004 15:35:37 -0700",-0700,2004-10-31 09:35:37-06:00,https://www.freelists.org/post/program-l/otiis...,None,https://www.freelists.org/post/program-l/otiis...,https://www.freelists.org/post/program-l/otiis...,1.0,NaN,39.0,2004-10-31 22:35:37,"thanks... I had thought of google, but.. well,..."
1,Re: ot--iis being weird. - program-l - FreeLists,"""Tyler Littlefield"" <compgeek13@xxxxxxxxx>",<program-l@xxxxxxxxxxxxx>,"Sun, 31 Oct 2004 15:37:53 -0700",-0700,2004-10-31 09:37:53-06:00,https://www.freelists.org/post/program-l/otiis...,None,https://www.freelists.org/post/program-l/otiis...,https://www.freelists.org/post/program-l/otiis...,1.0,NaN,39.0,2004-10-31 22:37:53,"lol... let me know if you make it, o... and yo..."
2,Re: ot--iis being weird. - program-l - FreeLists,"""Tyler Littlefield"" <compgeek13@xxxxxxxxx>",<program-l@xxxxxxxxxxxxx>,"Sun, 31 Oct 2004 15:53:33 -0700",-0700,2004-10-31 09:53:33-06:00,https://www.freelists.org/post/program-l/otiis...,https://www.freelists.org/post/program-l/otiis...,https://www.freelists.org/post/program-l/otiis...,https://www.freelists.org/post/program-l/otiis...,1.0,1.0,39.0,2004-10-31 22:53:33,well... if I get can not find server... google...
3,Re: List Move - Completed? - program-l - FreeL...,"""George Bell"" <george@xxxxxxxxxxxxxxxxxxx>","<program-l@xxxxxxxxxxxxxxxx>, <program-l@xxxxx...","Sun, 7 Nov 2004 17:42:11 -0000",-0000,2004-11-07 11:42:11-06:00,https://www.freelists.org/post/program-l/List-...,None,None,None,NaN,NaN,NaN,2004-11-07 17:42:11,"Hi all,\n\nFurther to my earlier message today..."
4,test - program-l - FreeLists,"""Corbett, James"" <James.Corbett@xxxxxxxxxxxxxxx>","""Program-L (E-mail)"" <Program-L@xxxxxxxxxxxxx>","Mon, 8 Nov 2004 08:35:31 -0500",-0500,2004-11-08 02:35:31-06:00,"https://www.freelists.org/post/program-l/test,...",https://www.freelists.org/post/program-l/Creat...,None,"https://www.freelists.org/post/program-l/test,...",NaN,1.0,10.0,2004-11-08 13:35:31,James M. Corbett\n\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56154,Re: css: responsive design question - program-...,Florian Beijers <florianbeijers@xxxxxxxxx>,program-l@xxxxxxxxxxxxx,None,None,NaT,https://www.freelists.org/post/program-l/css-r...,https://www.freelists.org/post/program-l/css-r...,https://www.freelists.org/post/program-l/css-r...,https://www.freelists.org/post/program-l/css-r...,17.0,1.0,28.0,NaT,"Hi,\n\nOk, most of this makes sense to me. Whe..."
56155,Re: python 2.7: say tools not speaking - progr...,Andreas Stefik <stefika@xxxxxxxxx>,program-l@xxxxxxxxxxxxx,None,None,NaT,https://www.freelists.org/post/program-l/pytho...,https://www.freelists.org/post/program-l/pytho...,https://www.freelists.org/post/program-l/pytho...,https://www.freelists.org/post/program-l/pytho...,21.0,1.0,27.0,NaT,"Yuma,\n\nIt will be once we're finished, yes. ..."
56156,Re: python 2.7: say tools not speaking - progr...,Andreas Stefik <stefika@xxxxxxxxx>,"program-l@xxxxxxxxxxxxx, Evan Pierzina <ekpier...",None,None,NaT,https://www.freelists.org/post/program-l/pytho...,None,https://www.freelists.org/post/program-l/pytho...,https://www.freelists.org/post/program-l/pytho...,21.0,NaN,27.0,NaT,Sounds good and feel free. If I recall correct...
56157,"Re: (No Date: Fri, 14 May 2021 18:04:02 -0400 ...","""Immigrant"" <dmarc-noreply@xxxxxxxxxxxxx> (Red...",<program-l@xxxxxxxxxxxxx>,None,None,NaT,https://www.freelists.org/post/program-l/No-Da...,https://www.freelists.org/post/program-l/No-Da...,https://www.freelists.org/post/program-l/no-su...,https://www.freelists.org/post/program-l/No-Da...,1.0,1.0,1.0,NaT,It shouldn't have been a zip archive. I downlo...
